In [31]:
import pandas as pd
import numpy as np
import tensorflow as tf


Load data

In [32]:
cf15= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/cf15.csv', low_memory=False, header=None)
gf25= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/gf25.csv', low_memory=False, header=None)
gm01= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/gm01.csv', low_memory=False, header=None)
nc46= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/nc46.csv', low_memory=False, header=None)
nc74= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/nc74.csv', low_memory=False, header=None)
nf11= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/nf11.csv', low_memory=False, header=None)
wc16= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/wc16.csv', low_memory=False, header=None)
wc41= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/wc41.csv', low_memory=False, header=None)
wo36= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/wo36.csv', low_memory=False, header=None)

In [33]:
val_1 = pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/go22.csv', low_memory=False, header=None)
val_2 = pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/kf15.csv', low_memory=False, header=None)
val_3 = pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/nf28.csv', low_memory=False, header=None)

In [34]:
frames =[cf15, nc46, nc74, nf11, wc16, wc41, wo36]
frames_2=[val_1, val_2]
data = pd.concat(frames)
val = pd.concat(frames_2)

Define data cleaning functions

In [68]:
def snip(data):
    #Snip unwanted characters 

    time= data.iloc[:,0]
    time=time.map(lambda x: str(x[6:]))
    time=time.map(lambda x: float(x))

    interface= data.iloc[:,-14]
    interface=interface.map(lambda x: str(x[15:]))
    interface=interface.map(lambda x: str(x[:-1]))
    interface=interface.map({'NI1A-WWW1':1, 'NI1B-WWW1':2, 'NI1B-WWW2':3, 'NI1A-WWW2':4 ,'NI1A-WWW3':5, 'NI1B-WWW3':6, 'NI1A-MPLS':7})

    jitter=data.iloc[:, -13]
    jitter=jitter.map(lambda x: str(x))
    jitter=jitter.map(lambda x: str(x[8:]))
    jitter=jitter.map(lambda x: str(x[:-1]))
    jitter=jitter.map(lambda x: float(x))

    latency= data.iloc[:,-12]
    latency=latency.map(lambda x: str(x))
    latency=latency.map(lambda x: str(x[9:]))
    latency=latency.map(lambda x: str(x[:-1]))
    latency=latency.map(lambda x: float(x))

    inband= data.iloc[:,-15]
    inband=inband.map(lambda x: str(x))
    inband=inband.map(lambda x: str(x[22:]))
    inband=inband.map(lambda x: str(x[:-5]))
    inband=inband.map(lambda x: float(x))

    outband= data.iloc[:, -5]
    outband=outband.map(lambda x: str(x))
    outband=outband.map(lambda x: x.split('"')[1])
    outband=outband.map(lambda x: str(x[:-4]))
    outband=outband.map(lambda x: float(x))

    packet=data.iloc[:,-4]
    packet=packet.map(lambda x: str(x))
    packet=packet.map(lambda x: str(x[12:]))
    packet=packet.map(lambda x: str(x[:-2]))
    packet=packet.map(lambda x: float(x))

    status=data.iloc[:,-2]
    status=status.map(lambda x: str(x))
    status=status.map(lambda x: str(x[8:]))
    status=status.map(lambda x: str(x[:-1]))

    status[:] = np.where(status=="up", 1,0)
    status=status.map(lambda x: float(x))
    status=status.rolling(20).mean()
    status=status.map(np.floor)

    train_init=pd.concat([time, interface, inband, outband, latency, packet, jitter, status], axis=1)
    train_init.columns=['Time', 'Interface', 'Inband', 'Outband', 'Latency', 'Packet', 'Jitter', 'Status']

    #Dedupp based on interfaces
    train=train_init[train_init.Interface != 'NI2A-WWW1']
    train=train[train.Interface != 'NI2B-WWW1']
    train=train[train.Interface != 'NI2A-WWW2']
    train=train[train.Interface != 'NI2B-WWW2']
    train=train[train.Interface != 'NI2A-WWW3']    
    train=train[train.Interface != 'NI2B-WWW3']
    train=train[train.Interface != 'NI2A-MPLS']
    train_label=train.filter(['Status'])
    train=train.drop(['Status'], axis=1)
    #train=train.sort_values(by=['Time'])
    train=train.reset_index()

    return train, train_label

In [69]:
train, labels= snip(data)

In [73]:
train.iloc[:,2].value_counts()

1.0    100361
2.0     79834
3.0     51918
5.0     47953
4.0     43224
6.0     17388
7.0     10078
Name: Interface, dtype: int64

In [25]:
def preproccess(data):
    train,train_label=snip(data)
    train=train.iloc[:-700, :]
    train=train.to_numpy()
    train_label=train_label.iloc[700:]
    return train, train_label


In [26]:
train, train_label = preproccess(data)
mean=train.mean(axis=0)
train-=mean
std = train.std(axis=0)
train/=std
test, test_label = preproccess(val)
test-=mean
test/=std

train_dataset = tf.data.Dataset.from_tensor_slices((train, train_label))
test_dataset = tf.data.Dataset.from_tensor_slices((test, test_label))

Build and train model

In [29]:
def get_model():
    model = tf.keras.Sequential([
            tf.keras.layers.Dense(10, activation='relu'),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(10, activation='relu'),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(10, activation='relu'),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(1, activation='sigmoid')])
    model.compile(optimizer='RMSprop',
            loss='BinaryCrossentropy',
            metrics=['accuracy'])    
    return model

In [30]:
model = get_model()       
history = model.fit(train, train_label, epochs=10, batch_size=128)

Epoch 1/10
5464/5464 [==============================] - 9s 2ms/step - loss: nan - accuracy: 0.4216
Epoch 2/10
2493/5464 [============>.................] - ETA: 6s - loss: nan - accuracy: 0.4207

KeyboardInterrupt: 

In [23]:
test_loss, test_acc =model.evaluate(test, test_label, verbose=2)
print('\nTest Accuracy:', test_acc)

3292/3292 - 2s - loss: 0.9951 - accuracy: 0.5778 - 2s/epoch - 733us/step

Test Accuracy: 0.5777961611747742
